In [2]:
import prepare_data
import pandas as pd

In [2]:
id2label, label2id, submission_file, test_ids_final, test_ocr_clean, train_ids_final, train_labels_final, train_set_ocr = prepare_data.read_files()

In [234]:
train_data = pd.read_csv('train_data_modified.csv')
test_data = pd.read_csv('test_data_modified.csv')

In [235]:
train_data = train_data[train_data['isPolish'] == 0]
test_data = test_data[test_data['isPolish'] == 0]

## Data Preparation

In [236]:
train_data

,Unnamed: 0.1,Unnamed: 0,File,Text ING,ID,Text Adam,Nrow,Text Adam pl,isPolish,isEnglish,isOther
1490,1490,1710,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,( Sptelipeyiię zako a o boga | | MISONE 2 A LN...,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,0,0,1
1650,1650,3596,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,contribułors - j MARIK MANU słetegahs hoc THOM...,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1,0
1651,1651,8555,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,00000 RL W O OPR PA e thiagwarm aid ferry that...,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,1,0
1652,1652,4794,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,1,0
1653,1653,10120,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,IMAGE NOT AVAILABLE ONLINE The material refere...,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9750,9750,7098,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,1,0
9751,9751,7042,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,1,0
9752,9752,4303,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,"A 41% KEOMAS ć 9 y ST USA. 0 PARK AVENUE, NEM ...",39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1,0
9753,9753,4455,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,"EP A — O o 7 p"" pzzą 0. Te fo dzi: 843 1082 e”...",47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,1,0


In [263]:
df = train_data.drop(columns=['isPolish','Unnamed: 0','isEnglish', 'Text Adam'])
df = df.drop(columns=["Unnamed: 0.1"])

df_val = test_data.drop(columns=['isPolish','Unnamed: 0','isEnglish', 'Text Adam'])
df_val = df_val.drop(columns=["Unnamed: 0.1"])
df

,File,Text ING,ID,Nrow,Text Adam pl,isOther
1490,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,1
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0
...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0


In [264]:
import re
from fuzzywuzzy import fuzz

string_column = "Text Adam pl"
df[string_column] = df[string_column].astype(str)
df['string_length'] = df[string_column].apply(lambda x: len(x))

# Add a column for the count of capital letters
df['capital_letters_count'] = df[string_column].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add a column for the count of numbers
df['numbers_count'] = df[string_column].apply(lambda x: sum(1 for c in x if c.isdigit()))
# Add a column for the count of question marks
df['question_marks_count'] = df[string_column].apply(lambda x: x.count('?'))
df['exclamation_marks_count'] = df[string_column].apply(lambda x: x.count('!'))
# Add a column for the count of currency signs
df['currency_signs_count'] = df[string_column].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))


# american phone regex
pattern = r"\b\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"

# american address regex
pattern1 = r"^\d+\s+([a-zA-Z]+\s+)+[a-zA-Z]+,\s*[A-Za-z]{2}\s*\d{5}(?:-\d{4})?$"

# Add a column for the flag indicating pattern match
df['flag_phone'] = df[string_column].apply(lambda x: 1 if re.search(pattern, x) else 0)

# Define the pattern to search for
pattern = r"\bi want it\b"

# Add a column for the flag indicating pattern match
df['flag_text'] = df[string_column].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)





df_val[string_column] = df_val[string_column].astype(str)
df_val['string_length'] = df_val[string_column].apply(lambda x: len(x))

# Add a column for the count of capital letters
df_val['capital_letters_count'] = df_val[string_column].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add a column for the count of numbers
df_val['numbers_count'] = df_val[string_column].apply(lambda x: sum(1 for c in x if c.isdigit()))
# Add a column for the count of question marks
df_val['question_marks_count'] = df_val[string_column].apply(lambda x: x.count('?'))
df_val['exclamation_marks_count'] = df_val[string_column].apply(lambda x: x.count('!'))
# Add a column for the count of currency signs
df_val['currency_signs_count'] = df_val[string_column].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))


# american phone regex
pattern = r"\b\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"

# american address regex
pattern1 = r"^\d+\s+([a-zA-Z]+\s+)+[a-zA-Z]+,\s*[A-Za-z]{2}\s*\d{5}(?:-\d{4})?$"

# Add a column for the flag indicating pattern match
df_val['flag_phone'] = df_val[string_column].apply(lambda x: 1 if re.search(pattern, x) else 0)

# Define the pattern to search for
pattern = r"\bi want it\b"

# Add a column for the flag indicating pattern match
df_val['flag_text'] = df_val[string_column].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)

df

,File,Text ING,ID,Nrow,Text Adam pl,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text
1490,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,1,434,84,20,0,0,0,0,0
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1723,463,18,1,0,0,0,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,2245,317,37,1,0,0,0,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,488,176,29,0,0,0,0,0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,231,30,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,302,18,16,0,0,0,0,0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,846,59,22,0,0,0,0,0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1259,234,63,0,0,3,1,0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,2069,257,39,1,1,1,0,0


In [265]:
import nltk
#import IProgress
import ipywidgets
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
tqdm.pandas()

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\wkosi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [266]:
def preprocess_text(text):
    # Tokenize the text
    #tokens = word_tokenize(str(text).lower())
    tokens = str(text).lower().split()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join the tokens back into a string
    #preprocessed_text = ' '.join(tokens)
    return tokens

In [267]:
email_keywords = ["original message", "cc:", "subject:", "re:", "attachments"]
letter_keywords = ["dear", "regards", "sincerely", "miss", "sir"]
invoices_keywords = ["invoice", "voucher", "amount","description","estimate","rate","payment","price"]
forms_keywords = ["form","request","report","confidential"]
memos_keywords = ["memo","memorandum"]
budget_keywords = [
    "budget", "period", "cost", "expenditure", "subtotal", "total"
]
specification_keywords = ["specification", "spec"]
scientific_report_keywords = ["abstract", "introduction", "method", "results","curriculum vitae","biographical sketch","redacted material"]
advertisment_keywords = ["smoking",  "cigarettes"]




all_keywords = email_keywords + letter_keywords + invoices_keywords + forms_keywords + memos_keywords + budget_keywords + specification_keywords + scientific_report_keywords + advertisment_keywords
keywords_counter = dict(zip(all_keywords, [[] for i in range(len(all_keywords))]))

In [181]:
from collections import Counter
counter =0
for index, row in df.iterrows():
    for keyword in all_keywords:
        if re.findall(keyword,row['Text Adam pl'].lower()):
            keywords_counter[keyword].append(row['ID'])

for key, value in keywords_counter.items():
    print(f"Key: {key}, {Counter(value)}")



Key: original message, Counter({2: 11, 4: 8})
Key: cc:, Counter({2: 60, 8: 29, 7: 12, 17: 2, 4: 2, 9: 2, 5: 1, 15: 1, 1: 1, 16: 1})
Key: subject:, Counter({2: 225, 8: 151, 7: 62, 4: 30, 14: 3, 13: 3, 16: 2, 9: 1, 1: 1, 12: 1})
Key: re:, Counter({2: 97, 8: 79, 7: 70, 17: 52, 4: 37, 13: 31, 16: 26, 15: 19, 9: 17, 6: 15, 1: 13, 0: 12, 14: 11, 12: 10, 3: 5, 5: 4})
Key: attachments, Counter({2: 19, 8: 6, 4: 3, 16: 1, 7: 1})
Key: dear, Counter({7: 230, 13: 20, 5: 17, 4: 14, 15: 12, 2: 10, 9: 10, 1: 7, 14: 5, 8: 5, 0: 1, 17: 1, 12: 1})
Key: regards, Counter({7: 23, 8: 11, 2: 10, 4: 9, 15: 2, 6: 1, 16: 1})
Key: sincerely, Counter({7: 85, 4: 5, 5: 2, 13: 2, 8: 1, 12: 1})
Key: miss, Counter({4: 78, 7: 41, 14: 39, 12: 39, 9: 33, 1: 18, 2: 16, 6: 16, 15: 15, 8: 10, 16: 10, 13: 10, 0: 5, 17: 5, 5: 1})
Key: sir, Counter({15: 60, 9: 32, 7: 26, 14: 23, 8: 22, 4: 17, 1: 14, 16: 12, 12: 12, 6: 11, 13: 11, 0: 8, 17: 8, 5: 5, 2: 5, 3: 2})
Key: invoice, Counter({6: 106, 1: 22, 4: 7, 8: 3, 7: 3, 2: 1})
Key:

## Model

In [268]:
df

,File,Text ING,ID,Nrow,Text Adam pl,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text
1490,570492ae-1a40-40cf-8d45-ed3591ea74ae,"nazwisko): NIP; I przez miasta, inazwisko)H?U ...",20,20,( Sptelipeyiie zako a o boga | | MISONE 2 A LN...,1,434,84,20,0,0,0,0,0
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1723,463,18,1,0,0,0,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,2245,317,37,1,0,0,0,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,488,176,29,0,0,0,0,0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,231,30,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,302,18,16,0,0,0,0,0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,846,59,22,0,0,0,0,0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1259,234,63,0,0,3,1,0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,2069,257,39,1,1,1,0,0


In [269]:
df2 = df[~df.ID.isin([10,11,18,19,20])]
df2_val = df_val[~df_val.ID.isin([10,11,18,19,20])]

In [271]:
for keyword in all_keywords:
    df2[keyword] = 0
    df2_val[keyword] = 0

for index, row in df2.iterrows():
    for keyword in all_keywords:
        df2.at[index, keyword] += len(re.findall(keyword,row[string_column].lower()))
        #df2.at[index, keyword] = 1 if fuzz.partial_ratio(keyword, row["Text Adam pl"].lower()) > 90 else 0

for index, row in df2_val.iterrows():
    for keyword in all_keywords:
        df2_val.at[index, keyword] += len(re.findall(keyword,row[string_column].lower()))

C:\Users\wkosi\AppData\Local\Temp\ipykernel_11436\111727474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[keyword] = 0


In [272]:
df2

,File,Text ING,ID,Nrow,Text Adam pl,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,...,spec,abstract,introduction,method,results,curriculum vitae,biographical sketch,redacted material,smoking,cigarettes
1650,1f8d02ff-2739-4d5b-8029-3d01700acb2e,contributors MARK MANN's Potograrhs have THOMA...,0,51,contribulors - j MARIK MANU sletegahs hoc THOM...,0,1723,463,18,1,...,0,0,0,0,0,0,0,0,1,0
1651,7b3ecce6-cea7-4378-af81-dab9434e8ca7,thing amnardiLmythat sj Sta adnoreanimaled ho ...,0,122,00000 RL W O OPR PA e thiagwarm aid ferry that...,0,2245,317,37,1,...,0,0,0,0,0,0,0,0,0,0
1652,2ac406e6-2fcd-45c1-989d-ee17fca95425,Catalina Marketing 365 West Passaic Strect Roc...,0,25,"s "" w EZIO ON PE TE FEB rare wd MO RR Gycle: 0...",0,488,176,29,0,...,0,0,0,0,0,0,0,0,0,0
1653,78a696be-d7b7-4cb7-a37f-ee3587f601e2,IMAGE NOT AVAILABLE ONLINE The material refere...,0,9,IMAGE NOT AVAILABLE ONLINE The material refere...,0,231,30,0,0,...,0,0,0,0,0,0,0,0,0,0
1654,1f3fbdca-c2be-44c9-b788-eed483a8e9ae,Currently sold only surgeon GeNEAal'S WarNing ...,0,31,SURGEON GENENAL'S WARNING; Smoking Currently s...,0,699,234,7,0,...,0,0,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,6e914249-5314-46c6-bf95-f30228fb2b83,To: Weber Patti Itterman From: Jackie Klecan D...,7,12,"To: Boug Weber 3 Paki iermat> "" Fromm: Jackie ...",0,302,18,16,0,...,0,0,0,0,0,0,0,0,0,0
9751,d3d18ddd-a110-443a-aa82-20b7864ba3b8,ZVSA Basehall IODAY Weexly Koith Cutler Publi...,7,28,"Keith Cutler Pdblsker, Bacebal Weekiy June 28,...",0,846,59,22,0,...,0,0,0,0,0,0,0,0,0,0
9752,2061e531-a091-4f40-937f-a7fa8bada187,"PHILIP MORRIS US.4. ICD PAAK AVENUE New York, ...",7,39,"A 41% KEOMAS c 9 y ST USA. 0 PARK AVENUE, NEM ...",0,1259,234,63,0,...,0,0,0,0,0,0,0,0,0,0
9753,39b58825-ae30-4376-b332-611b14d88d03,0ba Sil Eus Ovx 543 1052 M~ chro )50 352 M Lw;...,7,47,"EP A — O o 7 p"" pzza 0. Te fo dzi: 843 1082 e”...",0,2069,257,39,1,...,1,0,0,0,0,0,0,0,0,0


In [273]:
mapping = dict(zip([ 0,  1,  2,  3,  4 , 5,  6,  7,  8 , 9 ,12, 13, 14, 15, 16 ,17],[ 0,  1 , 2,  3 , 4 , 5 , 6 , 7,  8,  9 ,10 ,11 ,12, 13, 14 ,15] ))

In [274]:
df2["ID"]=df2["ID"].map(mapping).astype(int)
df2_val["ID"]=df2_val["ID"].map(mapping).astype(int)

C:\Users\wkosi\AppData\Local\Temp\ipykernel_11436\2915549792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["ID"]=df2["ID"].map(mapping).astype(int)


In [275]:
df2["capital_letters_count"] = df2["capital_letters_count"] / (df2["string_length"]+1)
df2["numbers_count"] = df2["numbers_count"] / (df2["string_length"]+1)

df2_val["capital_letters_count"] = df2_val["capital_letters_count"] / (df2_val["string_length"]+1)
df2_val["numbers_count"] = df2_val["numbers_count"] / (df2_val["string_length"]+1)

C:\Users\wkosi\AppData\Local\Temp\ipykernel_11436\3221848050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["capital_letters_count"] = df2["capital_letters_count"] / (df2["string_length"]+1)
C:\Users\wkosi\AppData\Local\Temp\ipykernel_11436\3221848050.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["numbers_count"] = df2["numbers_count"] / (df2["string_length"]+1)


In [280]:
y_train = df2["ID"]
X_train = df2.drop(columns=["File","Text ING", "ID", "Text Adam pl"])

y_test = df2_val["ID"]
X_val = df2_val.drop(columns=["File","Text ING", "ID", "Text Adam pl"])

In [281]:
X_train

,Nrow,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text,...,spec,abstract,introduction,method,results,curriculum vitae,biographical sketch,redacted material,smoking,cigarettes
1650,51,0,1723,0.268561,0.010441,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1651,122,0,2245,0.141140,0.016474,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1652,25,0,488,0.359918,0.059305,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1653,9,0,231,0.129310,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1654,31,0,699,0.334286,0.010000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,12,0,302,0.059406,0.052805,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9751,28,0,846,0.069658,0.025974,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9752,39,0,1259,0.185714,0.050000,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
9753,47,0,2069,0.124155,0.018841,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [282]:
X_val

,Nrow,isOther,string_length,capital_letters_count,numbers_count,question_marks_count,exclamation_marks_count,currency_signs_count,flag_phone,flag_text,...,spec,abstract,introduction,method,results,curriculum vitae,biographical sketch,redacted material,smoking,cigarettes
185,54,1,1371,0.450437,0.034985,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
186,25,0,622,0.065811,0.003210,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187,36,1,1095,0.470803,0.023723,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
188,26,0,478,0.304802,0.041754,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189,51,0,1912,0.192891,0.010978,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,43,0,1508,0.145129,0.038436,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1090,34,0,1497,0.140854,0.033378,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1091,47,0,1403,0.184473,0.094729,0,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1092,27,0,1164,0.095279,0.041202,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [285]:
X_train = X_train.drop(columns=["Nrow"])
X_val = X_val.drop(columns=["Nrow"])

In [317]:
from xgboost import XGBClassifier
model=XGBClassifier(random_state=42,
                    learning_rate=0.04, 
                    booster='gbtree',
                    max_depth=8
                    )
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [325]:
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve

y_pred = model.predict(X_train)

print('Train:')
print(f'Accuracy Score: {accuracy_score(y_train,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_train, y_pred)}')
# print(f'Area Under Curve: {roc_auc_score(y_train, y_pred, multi_class="ovr")}')
print(f'Recall score: {recall_score(y_train,y_pred, average="micro")}')
print(f"Precision score: {precision_score(y_train, y_pred, average='micro')}")

Train:
Accuracy Score: 0.7347368421052631
Confusion Matrix: 
[[366   5   5  27   4  71  10   1   0   6  16   2   1   1   0   3]
 [ 22 369   4  26   7  21  21   2   2   4   8   1   1   2   3   1]
 [  3   5 426   8   5  24   2   2   2   4   6   5   5   7   1   1]
 [ 11   0   2 474   3   9   2   0   0   1   2   0   0   0   2   0]
 [ 29   6   3   8 332  32  33   8   6   7   4  10   6   4   7  14]
 [ 61   9  15  47   6 375   6   9   0   6   7   7   0   1   1   1]
 [ 22  23   4   7  13  16 380   1   2   1   6   3   6   0   3   9]
 [  8   1  14  10  22  23   3 326  26   8  19   8  11   6   3   7]
 [ 12   5   7   5  21  31   7   6 348   8  12   7  13   9   9   2]
 [ 29   0   6   3   7  42   9   3   0 304  14   7   3  18   1   1]
 [ 26  11  13  47  10  33   9   3   5  17 334   3   3  10   4   3]
 [ 22  11   3  14   8  23  18   2   6  15  12 347   6   6   2   0]
 [  3   0   2   2   1   9   4   1   2   4   8   3 449  15   2   3]
 [  6   2   5   5   2  24   5   3   0  16   5   2   3 439   2   0]
 

In [327]:
y_pred_val = model.predict(X_val)

print('Validation:')
print(f'Accuracy Score: {accuracy_score(y_test,y_pred_val)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred_val)}')
# print(f'Area Under Curve: {roc_auc_score(y_train, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred_val, average="micro")}')
print(f"Precision score: {precision_score(y_test,y_pred_val, average='micro')}")

Validation:
Accuracy Score: 0.4718853362734289
Confusion Matrix: 
[[18  1  2  8  4  7  3  0  1  8  4  1  0  0  0  1]
 [ 6 22  0  3  4  3 10  0  2  1  1  1  0  0  2  1]
 [ 4  1 34  2  0  6  1  0  4  2  0  1  0  0  1  1]
 [ 7  3  0 33  1  4  1  0  0  4  4  0  0  0  0  0]
 [ 3  3  2  2 15  6  6  1  4  2  1  4  2  3  0  3]
 [ 6  5  2 13  1 26  3  0  0  1  2  2  0  0  0  0]
 [ 7 14  0  2  2  1 25  0  1  0  1  0  1  0  0  2]
 [ 1  1  2  1  1  4  0 23 11  1  4  0  1  0  4  2]
 [ 0  0  1  1  5  4  4  1 29  2  2  0  2  2  2  1]
 [ 4  2  0  0  1  3  2  0  1 15  4  3  1 13  1  1]
 [ 5  1  3  6  4  2  1  0  1  1 30  0  1  2  2  0]
 [ 4  3  0  2  2  7  2  2  0  2  2 25  1  0  0  4]
 [ 0  1  1  1  1  1  0  3  2  0  1  0 42  4  0  0]
 [ 0  0  0  1  0  2  1  0  1  8  1  0  2 38  2  2]
 [ 2  3  1  4  5  4  7  0  2  2  3  1  6  0 17  2]
 [ 1  2  1  1  3  2  1  0  0  0  0  0  5  0  1 36]]
Recall score: 0.4718853362734289
Precision score: 0.4718853362734289


In [336]:
model.save_model("xgboost_model.json")

In [337]:
model.feature_importances_

array([0.00742417, 0.01271543, 0.00814687, 0.00845753, 0.02266489,
       0.00380937, 0.00513199, 0.0096025 , 0.        , 0.00252382,
       0.01312162, 0.04970557, 0.00728863, 0.00171458, 0.06657067,
       0.01034316, 0.02406827, 0.00957546, 0.00372209, 0.04043392,
       0.02165402, 0.00550052, 0.0053095 , 0.0052465 , 0.00473225,
       0.0110388 , 0.00425212, 0.00813907, 0.00662897, 0.01125404,
       0.00583556, 0.06304845, 0.02144881, 0.00781697, 0.00956095,
       0.00606121, 0.00632486, 0.01006335, 0.0074658 , 0.05560736,
       0.01488873, 0.00498395, 0.01010423, 0.0071937 , 0.01172881,
       0.0913808 , 0.23287696, 0.02804419, 0.00857014, 0.00621876],
      dtype=float32)